In [1]:
ZONE_ATTRIBUTES = r""
NEW_STATION_FILE = r""
SOURCE_CACHE = r""
TARGET_CACHE = r""

In [2]:
import inro.modeller as m
mm = m.Modeller()
gghm_api = mm.module('gghm.internal.api_interface')

In [3]:
gghm_api.remove_jupyter_handler()

In [4]:
import pandas as pd
import numpy as np

In [5]:
zones = set(pd.read_csv(ZONE_ATTRIBUTES, index_col='taz').index)
stations = set(pd.read_csv(NEW_STATION_FILE, index_col='taz').index)
gateways = {9401, 9401, 9402, 9403, 9404, 9405, 9406, 9407, 9408, 9409, 9410, 9411, 9412, 9413, 9501, 9502, 9503, 9504}
all_zones = pd.Index(sorted(zones | stations | gateways))
source_butler = gghm_api.MatrixButler.connect(SOURCE_CACHE)

In [6]:
new_butler = gghm_api.MatrixButler.create(TARGET_CACHE, all_zones, 4000)
new_butler.zone_partition = source_butler.zone_partition

Copy over the matrices, expanding as we go

In [7]:
od_matrix_list = source_butler.query_type('od_trips')
pa_matrix_list = [uid for uid in source_butler.query_type('pa_trips') if 'all_modes' in uid]

In [8]:
%%time
source_mindex = pd.MultiIndex.from_product([source_butler.zone_system]*2)
new_mindex = pd.MultiIndex.from_product([all_zones]*2)

Wall time: 215 ms


In [9]:
%%time
indexer = new_mindex.get_indexer_for(source_mindex)

Wall time: 27.2 s


In [10]:
def expand_matrix(uid, matrix_type):
    source_matrix = source_butler.load_matrix(uid, tall=True).values
    
    new_matrix = pd.Series(0.0, index=new_mindex).astype('f4').values
    new_matrix[indexer] = source_matrix
    
    new_matrix.shape = len(all_zones), len(all_zones)
    new_matrix = pd.DataFrame(new_matrix, index=all_zones, columns=all_zones)
    new_butler.save_matrix(new_matrix, uid, type_name=matrix_type)
    print uid

In [11]:
%%time
for uid in od_matrix_list: expand_matrix(uid, 'od_trips')
for uid in pa_matrix_list: expand_matrix(uid, 'pa_trips')

trips.am_peak.truck.light
trips.midday.truck.light
trips.pm_peak.truck.light
trips.evening.truck.light
trips.overnight.truck.light
trips.am_peak.truck.medium
trips.midday.truck.medium
trips.pm_peak.truck.medium
trips.evening.truck.medium
trips.overnight.truck.medium
trips.am_peak.truck.heavy
trips.midday.truck.heavy
trips.pm_peak.truck.heavy
trips.evening.truck.heavy
trips.overnight.truck.heavy
trips.peak.active.walk
trips.peak.active.bike
trips.offpeak.active.walk
trips.offpeak.active.bike
trips.peak.transit.train
trips.offpeak.transit.train
trips.peak.transit.subway
trips.offpeak.transit.subway
trips.peak.transit.ztsa
trips.offpeak.transit.ztsa
trips.peak.transit.express_bus
trips.offpeak.transit.express_bus
trips.peak.transit.rapid_bus
trips.offpeak.transit.rapid_bus
trips.peak.transit.local_bus
trips.offpeak.transit.local_bus
trips.peak.transit.drive_to_and_from
trips.offpeak.transit.drive_to_and_from
trips.am_peak.auto1.toll
trips.am_peak.auto2.nt_nh
trips.am_peak.auto2.nt_hov
tri